In [ ]:
# Gadoev Rahmonali
# Задание №1
# Создайте датафрейм содержащий сумму и количество продаж треков в разбивке по странам и континентам.
# Создайте scatter plot показывающий соотношение суммы продаж и количества треков.
# Добавьте разделение по цвету на основе континента. Цвет Азии должен быть жёлтым, цвет Европы синим, цвет Северной Америки красным.
# Добавьте название диаграмме. Шрифт должен быть 30, цвет шрифта серый.
# Измените шрифт тиков (значения осей) на 14, цвет на серый.
## Задание №2
# Создайте датафрейм содержащий количество треков в разбивке по названиям жанров.
# Создайте столбчатую диаграмму. По оси X должны быть названия жанров, по оси Y - количество треков.
# Кастомизируйте диаграмму так, чтобы столбики всех жанров были серого цвета. За исключением жанров Jazz, Rock и Alternative & Punk. Эти жанры должны выделяться цветом. Цвета можете выбирать сами.
# Уберите легенду.
# Добавьте название диаграмме. Название должно быть курсивным.
# Фон диаграммы должен быть белым.

In [1]:
from connect import set_connection
from sqlalchemy import text
import pandas as pd
import matplotlib as plt
import plotly.express as px

In [2]:
country_to_continent = {
    'USA': 'North America',
    'Canada': 'North America',
    'Germany': 'Europe',
    'France': 'Europe',
    'UK': 'Europe',
    'Brazil': 'South America',
    'Australia': 'Oceania',
    # Добавьте остальные страны из вашей базы данных
}

# SQL-запрос для получения данных
query = """
    SELECT 
        i.billing_country AS country,
        COUNT(*) AS sales_count,
        SUM(i.total) AS sales_amount
    FROM invoice i
    GROUP BY i.billing_country
"""

# Получаем данные
with set_connection() as conn:
    df = pd.read_sql(query, conn)

# Добавляем континенты
df['continent'] = df['country'].map(country_to_continent)

# Группируем по континентам
continent_df = df.groupby('continent').agg({
    'sales_count': 'sum',
    'sales_amount': 'sum'
}).reset_index()

# Объединяем данные
final_df = pd.concat([
    df[['country', 'sales_count', 'sales_amount']],
    continent_df.rename(columns={'continent': 'country'})
], ignore_index=True)

# Сортировка и форматирование
final_df = final_df.sort_values(by='sales_amount', ascending=False)
final_df['sales_amount'] = final_df['sales_amount'].round(2)

In [3]:
final_df

,country,sales_count,sales_amount
25,North America,147,827.02
11,USA,91,523.06
24,Europe,63,351.58
21,Canada,56,303.96
10,France,35,195.10
14,Brazil,35,190.10
27,South America,35,190.10
19,Germany,28,156.48
12,United Kingdom,21,112.86
5,Czech Republic,14,90.24


In [48]:
# 2
query = """
    SELECT 
        i.invoice_id,
        i.total AS sales_amount,
        COUNT(il.track_id) AS tracks_count
    FROM invoice i
    JOIN invoice_line il ON i.invoice_id = il.invoice_id
    GROUP BY i.invoice_id, i.total
"""

with set_connection() as conn:
    df = pd.read_sql(query, conn)

fig = px.scatter(
    df,
    x='tracks_count',
    y='sales_amount',
    title='Соотношение суммы продаж и количества треков',
    labels={
        'tracks_count': 'Количество треков в заказе',
        'sales_amount': 'Сумма продажи ($)'
    },
    trendline='ols',
    color_discrete_sequence=['#636EFA'],
    hover_data=['invoice_id'],
    height=600
)
fig.show()
fig.write_html("sales_vs_tracks_scatter.html")


In [9]:
 # 3 exercise
country_to_continent = {
    'USA': 'North America',
    'Canada': 'North America',
    'Germany': 'Europe',
    'France': 'Europe',
    'UK': 'Europe',
    'Japan': 'Asia',
    'China': 'Asia',
    'India': 'Asia'
}
query = """
    SELECT 
        i.invoice_id,
        i.billing_country AS country,
        i.total AS sales_amount,
        COUNT(il.track_id) AS tracks_count
    FROM invoice i
    JOIN invoice_line il ON i.invoice_id = il.invoice_id
    GROUP BY i.invoice_id, i.total, i.billing_country
"""

with set_connection() as conn:
    df = pd.read_sql(query, conn)

df['continent'] = df['country'].map(country_to_continent)

fig = px.scatter(
    df,
    x='tracks_count',
    y='sales_amount',
    color='continent',
    title='Соотношение суммы продаж и количества треков по континентам',
    labels={
        'tracks_count': 'Количество треков в заказе',
        'sales_amount': 'Сумма продажи ($)',
        'continent': 'Континент'
    },
    color_discrete_map={
        'Asia': 'yellow',
        'Europe': 'blue',
        'North America': 'red'
    },
    category_orders={'continent': ['Asia', 'Europe', 'North America']},
    height=600,
    hover_data=['country']
)
fig.update_layout(
    plot_bgcolor='rgba(245,245,245,1)',
    legend=dict(
        title='Континент',
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=50, r=50, b=50, t=80)
)

fig.update_traces(marker=dict(size=8, opacity=0.7))

fig.show()

In [10]:
# 4 exercise
fig.update_layout(
    title={
        'text': "Соотношение суммы продаж и количества треков",
        'font': {
            'size': 30,
            'color': 'gray'
        }
    },
    plot_bgcolor='rgba(245,245,245,1)',
    margin=dict(l=50, r=50, b=50, t=100),
    legend=dict(
        title='Континент',
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig.update_traces(marker=dict(size=8, opacity=0.7))

In [11]:
# 5 exercise
fig.update_layout(
    title={
        'text': "Соотношение суммы продаж и количества треков",
        'font': {
            'size': 30,
            'color': 'gray'
        }
    },
    plot_bgcolor='rgba(245,245,245,1)',
    margin=dict(l=50, r=50, b=50, t=100),
    xaxis=dict(
        title='Количество треков в заказе',
        title_font=dict(size=14),
        gridcolor='lightgray',
        tickfont=dict(
            size=14,
            color='gray'
        )
    ),
    yaxis=dict(
        title='Сумма продажи ($)',
        title_font=dict(size=14),
        gridcolor='lightgray',
        tickfont=dict(
            size=14,
            color='gray'
        )
    ),
    legend=dict(
        title='Континент',
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

In [14]:
# задача № 2
# 1 exercise
query = """
    SELECT 
        g.name AS genre_name,
        COUNT(*) AS tracks_count
    FROM track t
    JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY g.name
    ORDER BY tracks_count DESC
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)

print(df)

            genre_name  tracks_count
0                 Rock          1297
1                Latin           579
2                Metal           374
3   Alternative & Punk           332
4                 Jazz           130
5             TV Shows            93
6                Blues            81
7            Classical            74
8                Drama            64
9             R&B/Soul            61
10              Reggae            58
11                 Pop            48
12          Soundtrack            43
13         Alternative            40
14         Hip Hop/Rap            35
15   Electronica/Dance            30
16         Heavy Metal            28
17               World            28
18    Sci Fi & Fantasy            26
19      Easy Listening            24
20              Comedy            17
21          Bossa Nova            15
22     Science Fiction            13
23       Rock And Roll            12
24               Opera             1


In [15]:
# 2 ex
fig = px.bar(
    df,
    x='genre_name',
    y='tracks_count',
    title='Распределение треков по жанрам',
    labels={'genre_name': 'Жанр', 'tracks_count': 'Количество треков'},
    color='genre_name',
    text='tracks_count'
)

fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor='rgba(255,255,255,1)',
    font=dict(color='#555555'),
    showlegend=False
)

fig.show()

In [ ]:
# 3 ex

query = """
    SELECT 
        g.name AS genre_name,
        COUNT(*) AS tracks_count
    FROM track t
    JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY g.name
    ORDER BY COUNT(*) DESC
"""

with set_connection() as conn:
    df = pd.read_sql(query, conn)

highlight_genres = ['Rock', 'Jazz', 'Alternative & Punk']
df['color_group'] = df['genre_name'].apply(
    lambda x: x if x in highlight_genres else 'Other'
)

color_map = {
    'Rock': 'red',
    'Jazz': 'green',
    'Alternative & Punk': 'yellow',
    'Other': 'grey'
}
fig = px.bar(
    df,
    x='genre_name',
    y='tracks_count',
    title='Распределение треков по жанрам',
    labels={'genre_name': 'Жанр', 'tracks_count': 'Количество треков'},
    color='color_group',
    color_discrete_map=color_map,
    category_orders={'genre_name': df['genre_name']},
    text='tracks_count',
    height=600)


fig.show()

In [39]:
# 5 ex
query = """
    SELECT 
        g.name AS genre_name,
        COUNT(*) AS tracks_count
    FROM track t
    JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY g.name
    ORDER BY COUNT(*) DESC
"""

with set_connection() as conn:
    df = pd.read_sql(query, conn)

highlight_genres = ['Rock', 'Jazz', 'Alternative & Punk']
df['color_group'] = df['genre_name'].apply(
    lambda x: x if x in highlight_genres else 'Other'
)

color_map = {
    'Rock': 'red',
    'Jazz': 'green',
    'Alternative & Punk': 'yellow',
    'Other': 'grey'
}
fig = px.bar(
    df,
    x='genre_name',
    y='tracks_count',
    title='Распределение треков по жанрам',
    labels={'genre_name': 'Жанр', 'tracks_count': 'Количество треков'},
    color='color_group',
    color_discrete_map=color_map,
    category_orders={'genre_name': df['genre_name']},
    text='tracks_count',
    height=600)

fig.update_layout(
    title={
        'text': '<i>Распределение треков по жанрам</i>',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {
            'size': 20,
            'color': '#333333'
        }
    })

fig.show()

In [47]:
# 6 ex
query = """
    SELECT 
        g.name AS genre_name,
        COUNT(*) AS tracks_count
    FROM track t
    JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY g.name
    ORDER BY COUNT(*) DESC
"""

with set_connection() as conn:
    df = pd.read_sql(query, conn)

highlight_genres = ['Rock', 'Jazz', 'Alternative & Punk']
df['color_group'] = df['genre_name'].apply(
    lambda x: x if x in highlight_genres else 'Other'
)

color_map = {
    'Rock': 'red',
    'Jazz': 'green',
    'Alternative & Punk': 'yellow',
    'Other': 'grey'
}
fig = px.bar(
    df,
    x='genre_name',
    y='tracks_count',
    title='Распределение треков по жанрам',
    labels={'genre_name': 'Жанр', 'tracks_count': 'Количество треков'},
    color='color_group',
    color_discrete_map=color_map,
    category_orders={'genre_name': df['genre_name']},
    text='tracks_count',
    height=600)

fig.update_layout(
    title={
        'text': '<i>Распределение треков по жанрам</i>',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20, 'color': 'red'}
    },
    plot_bgcolor='white',  # Белый фон области диаграммы
    paper_bgcolor='white', # Белый фон вокруг диаграммы
    xaxis_tickangle=-45,
    font=dict(color='black'),
    margin=dict(l=50, r=50, b=100, t=100))
     
fig.show() 
